### **Libraries**

In [1]:
import os
import openai
from openai import OpenAI

In [2]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY") # OpenAI API key

#### **Define a client**

In [3]:
# define an instance of OpenAI model
client = OpenAI(api_key=OPENAI_API_KEY)

#### **Audio transcription**

In [4]:
audio_file= open("audio.mp3", "rb")

In [5]:
speech_to_text = client.audio.transcriptions.create(
    model="whisper-1",
    file=audio_file,
)

In [6]:
speech_to_text.text

'There will be a meeting on the 21st of July at 5pm and the meeting will be about the capstone project.'

### **Audio to file**

### **Function calling**

In [8]:
system_prompt = "You are an assistant that creates a well crafted schedule from text."

In [9]:
schedule_tool = [
    {
        'type': 'function',
        'function': {
            'name': 'schedule_events',
            'parameters': {
                'type': 'object',
                'properties': {
                    'date': {
                        'type': 'string',
                        'description': 'The date of the event in the format YYYY-MM-DD'
                    },
                    'time': {
                        'type': 'string',
                        'description': 'The time of the event in the format HH:MM'
                    },
                    'event': {
                        'type': 'string',
                        'description': 'The name of the event'
                    },
                    'location': {
                        'type': 'string',
                        'description': 'The location of the event'
                    },
                    'description': {
                        'type': 'string',
                        'description': 'A description of the event'
                    },
                    'reminder': {
                        'type': 'string',
                        'description': 'The time before the event to send a reminder'
                    }
                }
            },
            'required': ['date', 'time', 'event'],
        },
        'instruction': system_prompt
    }
]